# Data Layer: Setting up SQLite  


First, let's create a SQLite database for the penguin dataset. SQLite is a lightweight, easy-to-use, serverless SQL database engine.

Create a new file named database.py.

In [ ]:
# install
!pip install mlflow --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
# database.py
import sqlite3
import pandas as pd
import numpy as np

# load data
data = pd.read_csv("https://github.com/allisonhorst/palmerpenguins/raw/5b5891f01b52ae26ad8cb9755ec93672f49328a8/data/penguins_size.csv")
data = data.dropna()

def init_db():
  # load the penguin dataset into a Pandas DataFrame
  df = data[['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'species_short']]

  # connect to the SQLite database
  conn = sqlite3.connect("penguin.db")

  # save the Pandas DataFrame to the SQLite database
  df.to_sql("penguin", conn, if_exists="replace", index=False)

  # close the connection to the SQLite database
  conn.close()

if __name__ == '__main__':
    init_db()

Run database.py to create the database and the penguin_data table.

In [ ]:
# import
import pandas as pd
import sqlite3

# connect to the SQLite database
conn = sqlite3.connect("penguin.db")

# read the list of tables using Pandas
tables_df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

# print the table names
for table_name in tables_df['name']:
    print(table_name)

# close the connection
conn.close()

penguin


# Business Layer: Training the Machine Learning Model

We'll use the scikit-learn library to train a machine learning model for Penguin classification. 

Create a new file named ml_model.py and paste the following code:

In [ ]:
# import
import pandas as pd
import sqlite3
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

# connect to the SQLite database
conn = sqlite3.connect("penguin.db")

# read data from a table using Pandas
data_df = pd.read_sql("SELECT * FROM penguin", conn)

def train_model():
    X = data_df.drop('species_short', axis=1)
    y = data_df['species_short']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    clf = SVC()
    clf.fit(X_train, y_train)

    with open("model.pkl", "wb") as f:
        pickle.dump(clf, f)

    return clf.score(X_test, y_test)

if __name__ == '__main__':
    accuracy = train_model()
    print(f"Model trained with accuracy: {accuracy}")

Model trained with accuracy: 0.7313432835820896


# Presentation Layer: HTML & CSS

Create a new folder named templates, and inside it, create a new file named index.html.


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Penguin Classification</title>
    <link rel="stylesheet" href="static/style.css">
</head>
<body>
    <h1>Penguin Classification</h1>
    <form action="/classify" method="post">
        <label for="culmen_length_mm">culmen_length:</label>
        <input type="number" step="0.1" id="culmen_length_mm" name="culmen_length_mm" required><br><br>
        <label for="culmen_depth_mm">culmen_depth:</label>
        <input type="number" step="0.1" id="culmen_depth_mm" name="culmen_depth_mm" required><br><br>
        <label for="flipper_length_mm">flipper_length:</label>
        <input type="number" step="0.1" id="flipper_length_mm" name="flipper_length_mm" required><br><br>
        <label for="body_mass_g">body_mass:</label>
        <input type="number" step="0.1" id="body_mass_g" name="body_mass_g" required><br><br>
        <input type="submit" value="Classify">
    </form>
    {% if prediction %}
    <h2>Prediction: {{ prediction }}</h2>
    {% endif %}
</body>
</html>


SyntaxError: ignored

Now, let's create a basic CSS file to style our app. 

Create a new folder named ***static***, and inside it, create a new file named ***style.css***.



In [ ]:
body {
    font-family: Arial, sans-serif;
    max-width: 600px;
    margin: 0 auto;
    padding: 20px;
}

input[type=number], input[type=submit] {
    width: 100%;
    padding: 5px;
    margin: 5px 0;
    box-sizing: border-box;
}

input[type=submit] {
    background-color: #4CAF50;
    color: white;
    cursor: pointer;
}

# Connecting everything with Flask

Create a new file named ***app.py*** and paste.

In [ ]:
# import
from flask import Flask, render_template, request, jsonify
import pickle
import sqlite3

app = Flask(__name__)

with open("model.pkl", "rb") as f:
    model = pickle.load(f)

@app.route("/", methods=["GET"])
def index():
    return render_template("index.html", prediction=None)

@app.route("/classify", methods=["POST"])
def classify():
    culmen_length_mm = float(request.form["culmen_length_mm"])
    culmen_depth_mm = float(request.form["culmen_depth_mm"])
    flipper_length_mm = float(request.form["flipper_length_mm"])
    body_mass_g = float(request.form["body_mass_g"])

    data = [[culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g]]
    prediction = model.predict(data)[0]

    # save the data to the database
    connection = sqlite3.connect("penguin.db")
    cursor = connection.cursor()
    cursor.execute("INSERT INTO penguin (culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, species_short) VALUES (?, ?, ?, ?, ?)",
                   (culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, prediction))
    connection.commit()
    connection.close()

    return jsonify({"prediction": prediction})


if __name__ == "__main__":
    app.run(debug=True, port=5006)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5006
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


# MLflow

Change the RandomForestClassifier in our business layer.

In [ ]:
# import
import mlflow
import mlflow.sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pandas as pd

# connect to the SQLite database
conn = sqlite3.connect("penguin.db")

# read data from a table using Pandas
data_df = pd.read_sql("SELECT * FROM penguin", conn)

def train_model():
    mlflow.set_experiment("penguin_Classification")
    X = data_df.drop('species_short', axis=1)
    y = data_df['species_short']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    clf = RandomForestClassifier()

    with mlflow.start_run():
        clf.fit(X_train, y_train)

        # log model parameters
        mlflow.log_param("n_estimators", clf.n_estimators)
        mlflow.log_param("criterion", clf.criterion)

        # log model performance metrics
        train_score = clf.score(X_train, y_train)
        test_score = clf.score(X_test, y_test)
        mlflow.log_metric("train_score", train_score)
        mlflow.log_metric("test_score", test_score)

        # save the model as an artifact
        mlflow.sklearn.log_model(clf, "model")

    return clf, test_score

if __name__ == '__main__':
    clf, accuracy = train_model()
    print(f"Model trained with accuracy: {accuracy}")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.sklearn.log_model(clf, "model")
    mlflow.sklearn.log_model(clf, "model", registered_model_name="penguin_model")
    mlflow.sklearn.save_model(clf, "penguin_model_3")

    # launch MLflow UI
    import os
    os.system("mlflow ui")

2023/04/18 16:53:54 INFO mlflow.tracking.fluent: Experiment with name 'penguin_Classification' does not exist. Creating a new experiment.


Model trained with accuracy: 1.0


Successfully registered model 'penguin_model'.
2023/04/18 16:54:04 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: penguin_model, version 1
Created version '1' of model 'penguin_model'.
